In [1]:
# python test_model.py model=iphone_orig dped_dir=dped/ test_subset=full iteration=all resolution=orig use_gpu=true

from scipy import misc
import numpy as np
import tensorflow as tf
from models import resnet
import utils
import os
import sys

# process command arguments
# phone, dped_dir, test_subset, iteration, resolution, use_gpu = utils.process_test_model_args(sys.argv)
phone = "iphone_orig"
dped_dir = 'dped/'
test_subset = 'full'
iteration = "orig"
resolution = "orig"
use_gpu = "true"

# get all available image resolutions
res_sizes = utils.get_resolutions()

# get the specified image resolution
# IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_SIZE = utils.get_specified_res(res_sizes, phone, resolution)

IMAGE_HEIGHT = 480
IMAGE_WIDTH = 854
IMAGE_SIZE = IMAGE_HEIGHT * IMAGE_WIDTH * 3

# disable gpu if specified
config = tf.ConfigProto(device_count={'GPU': 0}) if use_gpu == "false" else None

# create placeholders for input images
x_ = tf.placeholder(tf.float32, [None, IMAGE_SIZE])
x_image = tf.reshape(x_, [-1, IMAGE_HEIGHT, IMAGE_WIDTH, 3])

# generate enhanced image
enhanced = resnet(x_image)

/home/student1/anaconda3/envs/keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def extract_crop(image, resolution, ):

    if resolution == "orig":
        return image

    else:

        x_up = int((res_sizes[phone][1] - res_sizes[resolution][1]) / 2)
        y_up = int((res_sizes[phone][0] - res_sizes[resolution][0]) / 2)

        x_down = x_up + res_sizes[resolution][1]
        y_down = y_up + res_sizes[resolution][0]

        return image[y_up : y_down, x_up : x_down, :]

In [5]:
with tf.Session(config=config) as sess:

    test_dir = 'dped/endoscope/'
    test_photos = [f for f in os.listdir(test_dir) if os.path.isfile(test_dir + f)]

    if test_subset == "small":
        # use five first images only
        test_photos = test_photos[0:5]

    if phone.endswith("_orig"):

        # load pre-trained model
        saver = tf.train.Saver()
        saver.restore(sess, "models_orig/" + phone)

        for photo in test_photos:

            # load training image and crop it if necessary

            print("Testing original " + phone.replace("_orig", "") + " model, processing image " + photo)
            image = np.float16(misc.imresize(misc.imread(test_dir + photo), [IMAGE_HEIGHT, IMAGE_WIDTH])) / 255

            #image_crop = utils.extract_crop(image, resolution, phone, [IMAGE_HEIGHT, IMAGE_WIDTH])
            image_crop_2d = np.reshape(image, [1, IMAGE_SIZE])

            # get enhanced image

            enhanced_2d = sess.run(enhanced, feed_dict={x_: image_crop_2d})
            enhanced_image = np.reshape(enhanced_2d, [IMAGE_HEIGHT, IMAGE_WIDTH, 3])

            before_after = np.hstack((image, enhanced_image))
            photo_name = photo.rsplit(".", 1)[0]

            # save the results as .png images

            misc.imsave("visual_results/endoscope/iphone/" + phone + "_" + photo_name + "_enhanced.png", enhanced_image)
            misc.imsave("visual_results/endoscope/iphone/" + phone + "_" + photo_name + "_before_after.png", before_after)

    else:

        num_saved_models = int(len([f for f in os.listdir("models/") if f.startswith(phone + "_iteration")]) / 2)

        if iteration == "all":
            iteration = np.arange(1, num_saved_models) * 1000
        else:
            iteration = [int(iteration)]

        for i in iteration:

            # load pre-trained model
            saver = tf.train.Saver()
            saver.restore(sess, "models/" + phone + "_iteration_" + str(i) + ".ckpt")

            for photo in test_photos:

                # load training image and crop it if necessary

                print("iteration " + str(i) + ", processing image " + photo)
                image = np.float16(misc.imresize(misc.imread(test_dir + photo), [IMAGE_HEIGHT, IMAGE_WIDTH])) / 255

#                 image_crop = utils.extract_crop(image, resolution, phone, res_sizes)
                image_crop_2d = np.reshape(image, [1, IMAGE_SIZE])

                # get enhanced image

                enhanced_2d = sess.run(enhanced, feed_dict={x_: image_crop_2d})
                enhanced_image = np.reshape(enhanced_2d, [IMAGE_HEIGHT, IMAGE_WIDTH, 3])

                before_after = np.hstack((image, enhanced_image))
                photo_name = photo.rsplit(".", 1)[0]

                # save the results as .png images

                misc.imsave("visual_results_iteration/" + phone + "_" + photo_name + "_iteration_" + str(i) + "_enhanced.png", enhanced_image)
                misc.imsave("visual_results_iteration/" + phone + "_" + photo_name + "_iteration_" + str(i) + "_before_after.png", before_after)


INFO:tensorflow:Restoring parameters from models_orig/iphone_orig
Testing original iphone model, processing image video51.avi_20180416_193655.389.jpg


/home/student1/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/student1/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/home/student1/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:36: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/home/student1/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:37: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Testing original iphone model, processing image video51.avi_20180416_193704.923.jpg
Testing original iphone model, processing image video46.avi_20180416_192509.697.jpg
Testing original iphone model, processing image video46.avi_20180416_191613.997.jpg
Testing original iphone model, processing image video51.avi_20180416_193147.602.jpg
Testing original iphone model, processing image video46.avi_20180416_191941.289.jpg
Testing original iphone model, processing image video51.avi_20180416_193712.032.jpg
Testing original iphone model, processing image video05.avi_20180416_190915.180.jpg
Testing original iphone model, processing image video05.avi_20180416_191014.191.jpg
Testing original iphone model, processing image video46.avi_20180416_191830.422.jpg
Testing original iphone model, processing image video51.avi_20180416_193643.895.jpg
Testing original iphone model, processing image video46.avi_20180416_191329.242.jpg
Testing original iphone model, processing image video51.avi_20180416_193104.